In [2]:
import csv
import pandas as pd

In [3]:
df_hoorn = pd.read_json('data.json')
df_utrecht = pd.read_json('Utrecht_Ibabs.json')

In [13]:
print(df_hoorn.shape)
print(df_utrecht.shape)

(2490, 35)
(16315, 28)


In [23]:
print(df_hoorn.columns)
print(df_utrecht.columns)

Index(['id', 'dc_title', 'dc_source', 'dc_type', 'foi_classificaties',
       'dc_publisher', 'foi_publishedDate', 'dc_date_year', 'foi_files',
       'dc_description', 'foi_files_length', 'ibabs_Gerelateerde items',
       'ibabs_Toelichting', 'ibabs_Partijen', 'ibabs_Portefeuillehouder',
       'ibabs_Status', 'ibabs_Afgedaan', 'ibabs_Agendapunt', 'ibabs_Domein',
       'ibabs_Stemmen', 'ibabs_Stand van zaken', 'ibabs_Datum afgedaan',
       'ibabs_Medewerker', 'ibabs_Deadline', 'ibabs_Afzender',
       'ibabs_Omschrijving', 'ibabs_Onderwerp',
       'ibabs_Datum uiterlijke beantwoording', 'ibabs_Steller',
       'ibabs_Datum B&W', 'ibabs_Datum beantwoording', 'ibabs_Opmerking',
       'ibabs_Datum tussenbericht', 'ibabs_Datum ingekomen',
       'ibabs_Toezegging'],
      dtype='object')
Index(['id', 'dc_title', 'dc_source', 'dc_type', 'foi_classificaties',
       'foi_publishedDate', 'dc_date_year', 'foi_files', 'dc_description',
       'foi_files_length', 'ibabs_Kenmerk', 'ibabs_In

In [22]:
print(len([i for i in df_hoorn.columns if i[:5] == 'ibabs']))
print(len([i for i in df_utrecht.columns if i[:5] == 'ibabs']))

24
18


In totaal heeft de gemeente Hoorn 2490 rapporten en de gemeente Utrecht 16315. Na het scrapen van de ibabs pagina van al deze rapporten blijkt dat de gemeente Hoorn 24 verschillende ibabs attributen hebben en bij Utrecht zijn dit er slechts 18 terwijl ze veel meer rapporten hebben. 

p.s. dat Hoorn in deze data ook meer gewonen attributen heeft komt omdat ik daar de TOOI codes al had toegevoegd dus is verder niet van belang

In [20]:
for i in df_hoorn.columns:
    if i[:5] == 'ibabs' and i in df_utrecht.columns:
        print(i)

    else:
        if i[:5] != 'ibabs':
            print(i)

ibabs_Gerelateerde items
ibabs_Portefeuillehouder
ibabs_Afgedaan
ibabs_Deadline
ibabs_Omschrijving
ibabs_Onderwerp
ibabs_Datum beantwoording
ibabs_Opmerking


Dit zijn alle Ibabs attributen die overeenkomen tussen de beiden gemeenten. Dit lijken ook allemaal wel standaard termen te zijn die verwacht mogen worden bij de bestuurlijke informatie.

In [34]:
df_hoorn.iloc[:, 11:].notna().sum()

ibabs_Gerelateerde items                  90
ibabs_Toelichting                        576
ibabs_Partijen                          1431
ibabs_Portefeuillehouder                1357
ibabs_Status                             631
ibabs_Afgedaan                          1349
ibabs_Agendapunt                         977
ibabs_Domein                             226
ibabs_Stemmen                            363
ibabs_Stand van zaken                    443
ibabs_Datum afgedaan                     276
ibabs_Medewerker                         366
ibabs_Deadline                            82
ibabs_Afzender                           324
ibabs_Omschrijving                         1
ibabs_Onderwerp                            9
ibabs_Datum uiterlijke beantwoording     521
ibabs_Steller                            506
ibabs_Datum B&W                          162
ibabs_Datum beantwoording                605
ibabs_Opmerking                           17
ibabs_Datum tussenbericht                 80
ibabs_Datu

Dit is hoe vaak bepaalde ibabs items voorkomen bij de gemeente Hoorn. Aangezien Hoorn in totaal bijna 2500 rapporten heeft blijkt al gelijk dat het grootste deel van de attributen niet veelvuldig voorkomen. Het attribuut omschrijving komt bijvoorbeeld slechts 1 keer voor.

In [51]:
x = df_hoorn[df_hoorn['ibabs_Omschrijving'].notna()]
print(x['dc_source'].iloc[0])

https://hoorn.bestuurlijkeinformatie.nl/Reports/Item/865ac4dc-dc6e-4b4e-9d0f-278309d3193b


Dat is dus dit ene item waarbij het op een ander woord voor de titel lijkt.

Ik moet ook nog vermelden dat ik de bijlagen weg heb gelaten omdat deze al in de API staan, maar eigenlijk was er ook nog een bijlage(n) / bijlage(s) attribuut. Alleen waren er dus twee verschillende spellingen. 

In [36]:
df_utrecht.iloc[:, 10:].notna().sum()

ibabs_Kenmerk                 3929
ibabs_Indiener(s)             3951
ibabs_Mede-indieners            91
ibabs_Onderwerp                 78
ibabs_Beleidsveld             7311
ibabs_Portefeuillehouder     10513
ibabs_Deadline                  66
ibabs_Datum beantwoording     1527
ibabs_Afgedaan                 126
ibabs_Relatie met               77
ibabs_Thema                   3672
ibabs_Datum invoer               3
ibabs_Gerelateerde items       109
ibabs_Opmerking                357
ibabs_Datum wijziging          729
ibabs_Omschrijving             234
ibabs_Opmerkingen                9
ibabs_Inhoud                   212
dtype: int64

In Utrecht is het gemiddeld aantal attributen per rapport nog veel lager. Want hier zijn meer dan 16000 rapporten en eigenlijk alleen portefeuillehouder en beleidsveld komen hier redelijk vaak voor. De overige attributen komen maar in een klein deel van de gevallen voor. Een groot deel van de attributen komt hier ook maar minder dan 100 keer voor. Wat dus bij minder dan 1 procent van alle rapporten is.

In [75]:
df_temp = df_utrecht.iloc[:, 10:]

print(df_temp.notna().sum(axis=1).mean())

df_temp.notna().sum(axis=1).value_counts()


2.0223107569721117


2     5047
0     4823
1     2167
6     1621
4     1145
3      944
5      304
7      114
8       96
9       35
10      15
11       4
Name: count, dtype: int64

Hier is het aantal keer dat een rapport een bepaald aantal ibabs attributen heeft bij de gemeente Utrecht. Een rapport bij de gemeente Utrecht heeft gemiddeld maar 2 ibabs attributen. Verreweg de meeste rapporten hebben dan ook 2 of minder attributen. Waarschijnlijk dus vaak portefeuillehouder en beleidsveld en verder niks. Daarna komt 1 attribuut. Toch heeft ook 6 attributen een verrassend hoog aantal. Dit kan wellicht komen door de foi_classificaties en dat een bepaald type altijd 6 attributen aanhoudt. De rapporten met de meeste ibabs attributen hebben er hier 11, maar dat zijn er slechts 4. 

In [74]:
df_temp = df_hoorn.iloc[:, 11:]

print(df_temp.notna().sum(axis=1).mean())

df_temp.notna().sum(axis=1).value_counts()



4.4586345381526105


7     550
1     368
0     357
6     307
8     229
5     225
2     124
3     119
4      93
9      78
10     35
11      4
12      1
Name: count, dtype: int64

Bij de gemeente Hoorn is het duidelijk dat veel vaker ibabs attributen worden gebruikt. Het gemiddelde is hier ruim twee keer zo groot als bij Utrecht namelijk bijna 4.5 attributen per rapport. Het is hier veel zeldzamer dat er rapporten zijn zonder attributen, en 7 verschillende attributen komt hier het meest voor.


In [76]:
df_utrecht.foi_classificaties.value_counts()

foi_classificaties
Raadsbrieven              8426
Schriftelijke vragen      3989
RSS                       1390
Memo's                     945
Verslagen M&S              576
Verslagen S&R              406
Verslagen Vragenuur        233
Besluitenlijst B&W         191
Archief                     59
Vragen Coronavirus          51
Export                      30
Referendumcommissie         10
Coalitievorming 2022         5
Geheimhoudingsregister       4
Name: count, dtype: int64

In [77]:
df_hoorn.foi_classificaties.value_counts()

foi_classificaties
Moties                              651
RSS                                 450
Toezeggingen                        365
Artikel 36 vragen                   349
Ingekomen stukken - College         326
Besluitenlijst B&W                  309
Financiële documenten                35
Verordeningen Raad en Commissies      5
Name: count, dtype: int64

Hier zijn de verdelingen van die foi_classificaties. Dat deze niet bepaald uniform zijn was al bekend door een eedere analyse. Dus daar ga ik verder niet op in. Al komen RSS, en Besluitenlijst B&W wel bij beiden voor. 

Ik wil nu ook nog voor deze classificaties kijken hoe vaak deze een ibabs_attribuut gemiddeld voorkomt bij al deze classificaties. Want een Besluitenlijst heeft waarschijnlijk gemiddeld veel minder attributen dan een motie.

In [99]:
df_temp = df_hoorn.groupby('foi_classificaties').count()

(df_temp.iloc[:, 10:].sum(axis=1) / df_temp['id']).round(2)

foi_classificaties
Artikel 36 vragen                   7.14
Besluitenlijst B&W                  0.00
Financiële documenten               0.14
Ingekomen stukken - College         1.06
Moties                              6.14
RSS                                 3.94
Toezeggingen                        6.82
Verordeningen Raad en Commissies    0.00
dtype: float64

Bij Hoorn zien we dit gelijk al terug. De besluitenlijsten hebben eigenlijk nooit Ibabs attributen, terwijl de artikel 36 vragen, moties en de toezeggingen er gemiddeld meer dan 6 hebben. 

In [104]:
df_temp = df_utrecht.groupby('foi_classificaties').count()

(df_temp.iloc[:, 9:].sum(axis=1) / df_temp['id']).round(2)

foi_classificaties
Archief                   0.15
Besluitenlijst B&W        0.01
Coalitievorming 2022      0.00
Export                    2.23
Geheimhoudingsregister    0.75
Memo's                    1.78
RSS                       0.02
Raadsbrieven              1.30
Referendumcommissie       0.00
Schriftelijke vragen      4.93
Verslagen M&S             0.00
Verslagen S&R             0.00
Verslagen Vragenuur       1.82
Vragen Coronavirus        3.00
dtype: float64

Ook bij de gemeente Utrecht zijn grote verschillen te zien tussen de verschillende classificaties. Deze verschillen zijn echter wel een stuk minder groot, omdat eigenlijk alle classificaties weinig attributen hebben. Alleen de schriftelijke